In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.metrics import Precision, Recall, BinaryAccuracy, F1Score
import os
import cv2
import imghdr
from matplotlib import pyplot as plt
import numpy as np

# data_dir = 'data/people/'
data_dir = 'data/pets/'

img_extensions = ['jpg', 'jpeg', 'png', 'bmp' ]

# img = cv2.imread(os.path.join('data', 'people', 'happy', '_happy_jumping_on_beach-40815.jpg'))
img = cv2.imread(os.path.join('data', 'pets', 'dog', 'dogs-playing-556304.jpg'))

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
for img_dir in os.listdir(data_dir):
    if img_dir == '.DS_Store':
        continue
    for img_path in os.listdir(os.path.join(data_dir, img_dir)):
        print(img)
        try: 
            img = cv2.imread(img_path)
            tip = imghdr.what(img_path)
            if tip not in img_extensions:
                print("Image extentions not valid")
                os.remove(img_path)
        except Exception as e:
            print(f"There is a problem with [{img_path}]" )

# data = tf.keras.utils.image_dataset_from_directory('data/people')
data = tf.keras.utils.image_dataset_from_directory('data/pets')

data_iterator = data.as_numpy_iterator()

batch = data_iterator.next()
batch[0].shape
batch[1]
fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for i, img in enumerate(batch[0][0:4]):
    ax[i].imshow(img.astype(int))
    ax[i].title.set_text(batch[1][i])

# data = tf.keras.utils.image_dataset_from_directory('data/people/')
data = tf.keras.utils.image_dataset_from_directory('data/pets/')

# Scalling our data

# x = images, y = target variable (labels)
data = data.map(lambda x, y: (x/255, y))
scaled_data_iterator = data.as_numpy_iterator()
batch = scaled_data_iterator.next()

fig, ax = plt.subplots(ncols=4, figsize=(20, 20))
for i, img in enumerate(batch[0][0:4]):
    ax[i].imshow(img)
    ax[i].title.set_text(batch[1][i])
len(data)

train_size = int(len(data) * 0.7)
validation_size = int(len(data) * 0.2)
test_size = int(len(data) * 0.1)
train = data.take(train_size)
validation = data.skip(train_size).take(validation_size)
test = data.skip(train_size + validation_size).take(test_size)
model = Sequential()

model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

log_dir = 'logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
history = model.fit(train, epochs=20, validation_data=validation, callbacks=[tensorboard_callback])
history.history

fig = plt.figure()

plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()
fig = plt.figure()

plt.plot(history.history['accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

precision = Precision()
recall = Recall()
accuracy = BinaryAccuracy()
f1_score = F1Score()

for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    precision.update_state(y, yhat)
    recall.update_state(y, yhat)
    accuracy.update_state(y, yhat)
print(f'precision: {precision.result()}, Recall: {recall.result()}, Accuracy: {accuracy.result()}')
# img = cv2.imread('happy_test.jpg')
# img = cv2.imread('sad_test.jpg')
# img = cv2.imread('dog_test.jpg')

img = cv2.imread('cat_test.jpeg')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()
resize = tf.image.resize(img, (256,256))
plt.imshow((resize.numpy().astype(int)))
plt.show()

# np.expand_dims(resize, 0)
y_hat = model.predict(np.expand_dims(resize/255, 0))

if y_hat > 0.5: 
    print(f'Predicted class is a Dog')
else:
    print(f'Predicted class is Cat')
# if y_hat > 0.5: 
#     print(f'Predicted class is Sad')
# else:
#     print(f'Predicted class is Happy')
# model.save(os.path.join('models','peopleclassifier.h5'))

model.save(os.path.join('models','petclassifier.h5'))
new_model = load_model(os.path.join('models','petclassifier.h5'))
new_model.predict(np.expand_dims(resize/255, 0))